In [2]:
import numpy as np
import random
import pygame
import sys
import math
import tkinter as tk
from tkinter import simpledialog




LIGHT_ORANGE = (255, 183, 77) 
BLACK = (0, 0, 0)             
WHITE_C = (250, 250, 250)     
BLACK_C = (0, 0, 0)            

PLAYER = 0
AI = 1


EMPTY = 0           
PLAYER_PIECE = 1    
AI_PIECE = 2        




WINDOW_LENGTH = 6


def create_board():
    board = np.zeros((19,19))
    return board


def drop_piece(board, row, col, piece):
    board[row][col] = piece



def is_valid_location(board, col):
    return board[board.shape[0] - 1][col] == 0


def get_next_open_row(board, col):
    for r in range(board.shape[0]):
        if board[r][col] == 0:
            return r


def print_board(board):
    print(np.flip(board, 0))




    
def winning_move(board, piece):
    for c in range(board.shape[1] - WINDOW_LENGTH + 1):
        for r in range(board.shape[0]):
            if all(board[r][c + i] == piece for i in range(WINDOW_LENGTH)):
                return True
    
    for c in range(board.shape[1]):
        for r in range(board.shape[0] - WINDOW_LENGTH + 1):
            if all(board[r + i][c] == piece for i in range(WINDOW_LENGTH)):
                return True
    
    for c in range(board.shape[1] - WINDOW_LENGTH + 1):
        for r in range(board.shape[0] - WINDOW_LENGTH + 1):
            if all(board[r + i][c + i] == piece for i in range(WINDOW_LENGTH)):
                return True
    
    for c in range(board.shape[1] - WINDOW_LENGTH + 1):
        for r in range(WINDOW_LENGTH - 1, board.shape[0]):
            if all(board[r - i][c + i] == piece for i in range(WINDOW_LENGTH)):
                return True


def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER_PIECE
    if piece == PLAYER_PIECE:
        opp_piece = AI_PIECE

    if window.count(piece) == 6:
        score += 1000
    elif window.count(piece) == 5 and window.count(EMPTY) == 1:
        score += 50
    elif window.count(piece) == 4 and window.count(EMPTY) == 2:
        score += 10

    if window.count(opp_piece) == 5 and window.count(EMPTY) == 1:
        score -= 40

    return score




def score_position(board, piece):
    score = 0

    center_array = [int(i) for i in list(board[:, board.shape[1] // 2])]
    center_count = center_array.count(piece)
    score += center_count * 3

    for r in range(board.shape[0]):
        row_array = [int(i) for i in list(board[r, :])]
        for c in range(board.shape[1] - 5):
            window = row_array[c:c + WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    for c in range(board.shape[1]):
        col_array = [int(i) for i in list(board[:, c])]
        for r in range(board.shape[0] - 5):
            window = col_array[r:r + WINDOW_LENGTH]
            score += evaluate_window(window, piece)

    for r in range(board.shape[0] - 5):
        for c in range(board.shape[1] - 5):
            window = [board[r + i][c + i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    for r in range(board.shape[0] - 5):
        for c in range(board.shape[1] - 5):
            window = [board[r + 5 - i][c + i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)

    return score                      




def is_terminal_node(board):
    return winning_move(board, PLAYER_PIECE) or winning_move(board, AI_PIECE) or len(get_valid_locations(board)) == 0




def minimax(board, depth, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI_PIECE):
                return (None, 100000000000000)
            elif winning_move(board, PLAYER_PIECE):
                return (None, -10000000000000)
            else:
                return (None, 0)
        else:
            return (None, score_position(board, AI_PIECE))

    if maximizingPlayer:
        value = -math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, AI_PIECE)
            new_score = minimax(b_copy, depth - 1, False)[1]
            if new_score > value:
                value = new_score
                column = col
        return column, value

    else:
        value = math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, PLAYER_PIECE)
            new_score = minimax(b_copy, depth - 1, True)[1]
            if new_score < value:
                value = new_score
                column = col
        return column, value



def get_valid_locations(board):
    valid_locations = []
    for col in range(board.shape[1]):
        if is_valid_location(board, col):
            valid_locations.append(col)
    return valid_locations


def pick_best_move(board, piece):
    valid_locations = get_valid_locations(board)
    best_score = -10000
    best_col = random.choice(valid_locations)
    for col in valid_locations:
        row = get_next_open_row(board, col)
        temp_board = board.copy()
        drop_piece(temp_board, row, col, piece)
        score = score_position(temp_board, piece)
        if score > best_score:
            best_score = score
            best_col = col

    return best_col




def draw_board(board):
    # Dynamically adjust circle size based on the board size
    circle_radius = int(min(SQUARESIZE / 2 - 5, height / (2 * board.shape[0])))

    for c in range(board.shape[1]):
        for r in range(board.shape[0]):
           pygame.draw.rect(screen, LIGHT_ORANGE, (c * SQUARESIZE, r * SQUARESIZE  , SQUARESIZE, SQUARESIZE))
           pygame.draw.rect(screen, BLACK, (c * SQUARESIZE, r * SQUARESIZE , SQUARESIZE, SQUARESIZE),1) 

    for c in range(board.shape[1]):
        for r in range(board.shape[0]):
            if board[r][c] == PLAYER_PIECE:
               pygame.draw.circle(screen, WHITE_C, (int(c * SQUARESIZE + SQUARESIZE / 1), height - int(r * SQUARESIZE + SQUARESIZE / 1)), circle_radius)   
            elif board[r][c] == AI_PIECE:
                pygame.draw.circle(screen, BLACK_C, (int(c * SQUARESIZE + SQUARESIZE / 1), height - int(r * SQUARESIZE + SQUARESIZE / 1)), circle_radius)

    pygame.display.update()





board = create_board()

game_over = False

print_board(board)

pygame.init()

SQUARESIZE = 40

width = board.shape[1] * SQUARESIZE
height = (board.shape[0] ) * SQUARESIZE



size = (width, height)

RADIUS = int(SQUARESIZE / 2 - 5)

screen = pygame.display.set_mode(size)

draw_board(board)

pygame.display.update()

myfont = pygame.font.SysFont("monospace", 75)



turn = PLAYER            
player_moves = 0         
max_moves_per_turn = 2   

while not game_over:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            game_over = True  
        if event.type == pygame.MOUSEBUTTONDOWN:
            pygame.draw.rect(screen, BLACK, (0, 0, width, SQUARESIZE))
            posx = event.pos[0]
            col = int((posx / SQUARESIZE))
       
            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, PLAYER_PIECE)
                player_moves += 1

                if player_moves == 1:
                    turn = AI
                elif player_moves == max_moves_per_turn:
                    turn = PLAYER
                    player_moves = 0  

                if winning_move(board, PLAYER_PIECE):
                    label = myfont.render("Player 1 wins!!", 1, WHITE_C)
                    screen.blit(label, (40, 10))
                    game_over = True
                elif len(get_valid_locations(board)) == 0:
                    label = myfont.render("It's a draw!", 1, (255, 255, 255))
                    screen.blit(label, (40, 10))
                    game_over = True

                print_board(board)
                draw_board(board)

    if turn == AI and not game_over:
        for _ in range(max_moves_per_turn):
            col, minimax_score = minimax(board, 2, True)

            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, AI_PIECE)

            if winning_move(board, AI_PIECE):
                label = myfont.render("AI wins!!", 1, BLACK_C)
                screen.blit(label, (40, 10))
                game_over = True
            elif len(get_valid_locations(board)) == 0:
                label = myfont.render("It's a draw!", 1, (255, 255, 255))
                screen.blit(label, (40, 10))
                game_over = True

            print_board(board)
            draw_board(board)

        turn = PLAYER

    if game_over:
        pygame.time.delay(2000)  

    pygame.time.Clock().tick(30)

pygame.quit()
sys.exit()

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

SystemExit: 

C:\Users\maham\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
